First:
- go to the terminal
- go to the `4_tuning_networks` folder, use `cd` to get there, or `z` to get there faster.
- if you are planning on closing the laptop to let this run by itself, start a tmux session by giving the command `tmux`. You can name the session with `ctrl+b $`, leave the tmux with `ctrl+b` d, access it again with `tmux a -t 0` or replace the 0 with the name you gave it. See [tmux](https://github.com/tmux/tmux/wiki/Getting-Started) for more info.
- Activate the `.venv`, and run `python hypertune.py`
- In the hypertune.py file, a tune_dir is specified: `models/ray`. We will check the contents of that folder after the hypertune finished. You can also use tensorboard to check the results. 
- this will take some time. I clocked 1901 seconds (30 minutes) on the VM

 0.0992908 |           119 |            3 |     26 |         576.744  |    0 |   0.942187

In [5]:
from pathlib import Path
tune_dir = Path("models/ray").resolve()
tune_dir.exists()

True

In [6]:
tunelogs = [d for d in tune_dir.iterdir()]
tunelogs.sort()
latest = tunelogs[-1]
latest

WindowsPath('C:/Users/dilek/Desktop/Advanced_AI_Applications_WS24-25_MADS_HSRW/notebooks/4_tuning_networks/models/ray/tune_model_2024-11-18_14-30-24')

In [7]:
from ray.tune import ExperimentAnalysis
import ray
ray.init(ignore_reinit_error=True)


analysis = ExperimentAnalysis(latest)


2024-11-24 11:29:19,195	INFO worker.py:1582 -- Calling ray.init() again after it has already been called.
2024-11-24 11:29:19,206	WARNING experiment_analysis.py:180 -- Failed to fetch metrics for 18 trial(s):
- tune_model_44781_00006: FileNotFoundError('Could not fetch metrics for tune_model_44781_00006: both result.json and progress.csv were not found at C:/Users/dilek/Desktop/Advanced_AI_Applications_WS24-25_MADS_HSRW/notebooks/4_tuning_networks/models/ray/tune_model_2024-11-18_14-30-24/tune_model_44781_00006_6_hidden_size=293,num_layers=2_2024-11-18_14-30-24')
- tune_model_44781_00005: FileNotFoundError('Could not fetch metrics for tune_model_44781_00005: both result.json and progress.csv were not found at C:/Users/dilek/Desktop/Advanced_AI_Applications_WS24-25_MADS_HSRW/notebooks/4_tuning_networks/models/ray/tune_model_2024-11-18_14-30-24/tune_model_44781_00005_5_hidden_size=280,num_layers=7_2024-11-18_14-30-24')
- tune_model_44781_00009: FileNotFoundError('Could not fetch metrics 

So, we find some info:

In [4]:
analysis.results_df.columns


Index([], dtype='object')

Let's focus on the parameters we wanted to tune.

In [ ]:
import plotly.express as px

plot = analysis.results_df
select = ["Accuracy", "config/hidden_size", "config/dropout", "config/num_layers"]
p = plot[select].reset_index().dropna()


Let's sort by accuracy

In [ ]:
p.sort_values("Accuracy", inplace=True)

Make a parallel plot

In [ ]:
px.parallel_coordinates(p, color="Accuracy")


In [ ]:
import seaborn as sns

sns.scatterplot(data=p, x="config/hidden_size", y="config/num_layers", hue="Accuracy", palette="coolwarm")

In [ ]:
import matplotlib.pyplot as plt
cmap = sns.cubehelix_palette(as_cmap=True)
sns.scatterplot(data=p, x="config/hidden_size", y="config/num_layers", hue="Accuracy", palette="coolwarm")
sns.kdeplot(data=p, x="config/hidden_size", y="config/num_layers", cmap=cmap)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1)

Get the best trial

In [ ]:
analysis.get_best_trial(metric="test_loss", mode="min")


The top ten

In [ ]:
p[-10:]


Or the best config

In [ ]:
analysis.get_best_config(metric="Accuracy", mode="max")
